# Prevair

Ce notebook sert à préparer les données des collectivités entre 20K et 50K habitants.

On va combiner des informations de ces sources :

https://www.data.gouv.fr/fr/datasets/mise-a-disposition-de-donnees-de-qualite-de-lair-sur-la-france-www-prevair-org-1/

Ce fichier compressé comprend deux propriétés racine :

* metadata : données calculées
* records : données extraites (enregistrements)

Les métadonnées comprennent la date, la BBOX et les minima et maxima de chaque type de prévision.

Chaque enregistrement comprend la totalité des prévisions pour une position géographique donnée, ainsi que les attributs lon et lat.

In [1]:
# Données prevair 
# https://www.data.gouv.fr/fr/datasets/mise-a-disposition-de-donnees-de-qualite-de-lair-sur-la-france-www-prevair-org-1/

# prev = pd.read_json('data/air/prevair-previsions-20180810.json')
# erreur car il y a de champs imbriqués

In [1]:
import pandas as pd
import numpy as np
import json
import requests

In [3]:
#d = json.loads(requests.get('http://localhost/www/energie/data/air/prevair-previsions-20180810.json').text)
d = json.loads(requests.get('http://localhost/www/energie/data/air/previsions-20180921.json').text)
list(d.keys())


['metadata', 'records']

In [4]:
meta = pd.DataFrame(d['metadata'])
meta.head(3)

,NO2-MAXJ0,NO2-MAXJ1,NO2-MAXJ2,NO2-MAXJ3,NO2-MOYJ0,NO2-MOYJ1,NO2-MOYJ2,NO2-MOYJ3,O3-MAXJ0,O3-MAXJ1,...,PM25-MAXJ3,PM25-MOYJ0,PM25-MOYJ1,PM25-MOYJ2,PM25-MOYJ3,date,maxLat,maxLon,minLat,minLon
maxValue,109.01,77.04,89.00,91.79,58.81,42.41,41.74,60.16,158.03,134.13,...,60.94,32.90,42.80,35.11,35.62,2018-09-21,52.289,11.172,40.508,-5.484
minValue,9.50,10.48,0.99,9.90,3.45,2.36,0.00,2.17,59.87,56.85,...,0.00,4.73,2.88,5.99,6.56,2018-09-21,52.289,11.172,40.508,-5.484


In [5]:
records = pd.DataFrame(d['records'])
records.head(3)

,NO2-MAXJ0,NO2-MAXJ1,NO2-MAXJ2,NO2-MAXJ3,NO2-MOYJ0,NO2-MOYJ1,NO2-MOYJ2,NO2-MOYJ3,O3-MAXJ0,O3-MAXJ1,...,PM25-MAXJ0,PM25-MAXJ1,PM25-MAXJ2,PM25-MAXJ3,PM25-MOYJ0,PM25-MOYJ1,PM25-MOYJ2,PM25-MOYJ3,lat,lon
0,22.05,21.05,13.17,25.04,8.80,7.37,6.43,9.22,97.16,91.51,...,24.83,23.07,24.71,24.53,15.90,16.89,18.49,17.82,40.508,-5.484
1,22.09,21.10,13.18,25.07,8.80,7.38,6.42,9.20,97.16,91.63,...,24.94,23.21,24.77,24.45,16.01,17.00,18.47,17.88,40.508,-5.453
2,22.31,21.17,13.20,25.12,8.85,7.40,6.43,9.20,97.12,91.78,...,25.02,23.31,24.83,24.17,16.05,17.05,18.44,17.91,40.508,-5.422


In [6]:
records.columns

Index(['NO2-MAXJ0', 'NO2-MAXJ1', 'NO2-MAXJ2', 'NO2-MAXJ3', 'NO2-MOYJ0',
       'NO2-MOYJ1', 'NO2-MOYJ2', 'NO2-MOYJ3', 'O3-MAXJ0', 'O3-MAXJ1',
       'O3-MAXJ2', 'O3-MAXJ3', 'O3-MOYJ0', 'O3-MOYJ1', 'O3-MOYJ2', 'O3-MOYJ3',
       'PM10-MAXJ0', 'PM10-MAXJ1', 'PM10-MAXJ2', 'PM10-MAXJ3', 'PM10-MOYJ0',
       'PM10-MOYJ1', 'PM10-MOYJ2', 'PM10-MOYJ3', 'PM25-MAXJ0', 'PM25-MAXJ1',
       'PM25-MAXJ2', 'PM25-MAXJ3', 'PM25-MOYJ0', 'PM25-MOYJ1', 'PM25-MOYJ2',
       'PM25-MOYJ3', 'lat', 'lon'],
      dtype='object')

In [7]:
records = records.drop(['NO2-MAXJ0', 'NO2-MAXJ1', 'NO2-MAXJ2', 'NO2-MAXJ3',
       'NO2-MOYJ1', 'NO2-MOYJ2', 'NO2-MOYJ3', 'O3-MAXJ0', 'O3-MAXJ1',
       'O3-MAXJ2', 'O3-MAXJ3', 'O3-MOYJ1', 'O3-MOYJ2', 'O3-MOYJ3',
       'PM10-MAXJ0', 'PM10-MAXJ1', 'PM10-MAXJ2', 'PM10-MAXJ3', 
       'PM10-MOYJ1', 'PM10-MOYJ2', 'PM10-MOYJ3', 'PM25-MAXJ0', 'PM25-MAXJ1',
       'PM25-MAXJ2', 'PM25-MAXJ3', 'PM25-MOYJ1', 'PM25-MOYJ2',
       'PM25-MOYJ3'], axis=1)
records = records.rename(columns={'NO2-MOYJ0': 'no2','O3-MOYJ0': 'o3','PM10-MOYJ0': 'pm10', 
                                  'PM25-MOYJ0': 'pm25', 'lat':'latitude', 'lon':'longitude'})
records.head(3)

,no2,o3,pm10,pm25,latitude,longitude
0,8.80,60.64,19.61,15.90,40.508,-5.484
1,8.80,61.10,19.72,16.01,40.508,-5.453
2,8.85,62.18,19.76,16.05,40.508,-5.422


### to CSV pour stockage

In [8]:
records.to_csv('data/air/prevair-201-09-21.csv')

### Re-open avec dtpe str

In [9]:
# Lors de la insertion SQL on avait une erreur de transformation np.float ... (?)
# on ouvre tout en str
#records = pd.read_csv('data/air/prevair.csv', dtype=str)
records = pd.read_csv('data/air/prevair-201-09-21.csv', dtype=str)
records.head(3)

,Unnamed: 0,no2,o3,pm10,pm25,latitude,longitude
0,0,8.8,60.64,19.61,15.9,40.508,-5.484
1,1,8.8,61.1,19.72,16.01,40.508,-5.453
2,2,8.85,62.18,19.76,16.05,40.508,-5.422


# To BDD

In [10]:
import pymysql.cursors

connection = pymysql.connect( host='localhost',
                              user='root',
                              passwd='root',
                              db='energie',
                              charset='utf8mb4',
                              cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()


In [11]:
with connection.cursor() as cursor:
    
    sql = ("INSERT INTO `prevair_moyJ0` "
           "( no2, o3, pm10, pm25, latitude, longitude )"
           "VALUES ( %s, %s, %s, %s, %s, %s )")

    for i, r in records.iterrows():
        cursor.execute(sql, 
                       (r['no2'], r['o3'], r['pm10'], r['pm25'], 
                        r['latitude'],r['longitude'] )
                      )

connection.commit() 